# LAB34: Rag Evaluation & RAGas

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


En este notebook veremos estos puntos:
- 🧪 Creación de Datos Sintéticos: Entender el proceso y la importancia de generar datos sintéticos para la evaluación RAG.
- 🛠️ Utilizando la Herramienta Ragas: Aprender cómo usar Ragas para una evaluación completa del rendimiento del modelo RAG a través de varias métricas.
- 🔍 Impacto de los Métodos de Recuperación: Explorar cómo diferentes enfoques de recuperación influyen en la efectividad y precisión de los modelos RAG.
- 💡 Aplicación Práctica: Aplicar estos conceptos a través de ejemplos y ejercicios para consolidar la comprensión y las habilidades en la evaluación RAG.


In [1]:
!pip install -qU langchain openai ragas arxiv pymupdf chromadb tiktoken accelerate bitsandbytes datasets sentence_transformers FlagEmbedding ninja  tqdm rank_bm25 transformers

In [ ]:
!pip install -U flash_attn --no-build-isolation

## Creamos nuestro RAG Pipeline

Primero crearemos nuestro pipeline de RAG para poder después evaluarlo.

Obtenemos de ArxivLoader algunos artículos que hablan sobre RAG:

- Evaluation Metrics in the Era of GPT-4: Reliably Evaluating Large Language Models on Sequence to Sequence Tasks
- A Survey on Evaluation of Large Language Models
- An Evaluation on Large Language Model Outputs: Discourse and Memorization
- A Closer Look into Automatic Evaluation Using Large Language Models
- Large Language Models are Not Yet Human-Level Evaluators for Abstractive Summarization
- ARES: An Automated Evaluation Framework for Retrieval-Augmented Generation Systems
- RaLLe: A Framework for Developing and Evaluating Retrieval-Augmented Large Language Models
- Benchmarking Large Language Models in Retrieval-Augmented Generation
- Evaluating the Effectiveness of Retrieval-Augmented Large Language Models in Scientific Document Reasoning


In [4]:
from langchain.document_loaders import ArxivLoader
from langchain.document_loaders.merge import MergedDataLoader

papers = ["2310.13800", "2307.03109", "2304.08637", "2310.05657", "2305.13091", "2311.09476", "2308.10633", "2309.01431", "2311.04348"]

docs_to_merge = []

for paper in papers:
    loader = ArxivLoader(query=paper)
    docs_to_merge.append(loader)

all_loaders = MergedDataLoader(loaders=docs_to_merge)
all_docs = all_loaders.load()

In [ ]:
!pip install langchain pypdf pytube youtube-transcript-api openai langchain_experimental langchain_openai

In [6]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders.merge import MergedDataLoader

# Ruta a la carpeta que contiene los PDFs
#pdf_folder = "/content/drive/MyDrive/Colab Notebooks/Proyecto Final/Pdfs"
pdf_folder = "./Pdfs"

# Obtener la lista de archivos PDF en la carpeta
pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.endswith(".pdf")]

# Cargar todos los PDFs
docs_to_merge = []

for pdf in pdf_files:
    loader = PyPDFLoader(pdf)
    docs_to_merge.append(loader)

all_loaders = MergedDataLoader(loaders=docs_to_merge)
all_docs = all_loaders.load()


In [ ]:
for doc in all_docs:
  print(doc.metadata)

Ahora crearemos la Base de Datos Vectorial con los documentos que hemos recuperado.

Usaremos [ChromaDB](https://www.trychroma.com/) como base de datos y [BAAI/bge-large-en-v1.5](https://huggingface.co/BAAI/bge-large-en-v1.5) como modelo de embedding.


In [8]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-large-en-v1.5"
encode_kwargs = {'normalize_embeddings': True}

hf_bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=512,
                                               chunk_overlap = 128,
                                               length_function=len)
docs = text_splitter.split_documents(all_docs)
vectorstore = Chroma.from_documents(docs, hf_bge_embeddings)

<ipython-input-8-134bb17aa32e>:8: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_bge_embeddings = HuggingFaceBgeEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [9]:
len(docs)

1298

In [10]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k" : 5})

In [ ]:
###
relevant_docs = base_retriever.get_relevant_documents("What are the challenges in evaluating Retrieval Augmented Generation pipelines?")

In [11]:
relevant_docs = base_retriever.get_relevant_documents("Cómo se gestionan las sustancias químicas?")

<ipython-input-11-9c6821c8e7ed>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = base_retriever.get_relevant_documents("Cómo se gestionan las sustancias químicas?")


In [14]:
relevant_docs = base_retriever.get_relevant_documents("¿Qué es el consumo sustentable?")

In [15]:
len(relevant_docs)

5

In [16]:
for doc in relevant_docs:
  print(doc.page_content)
  print('\n')

producción, intercambio y consumo sostenible de bienes y servicios, con el fin de evitar o reducir la
contaminación y optimizar el uso del recurso natural.
3. Desarrollo Sostenible. Es el proceso mediante el cual, de manera dinámica, se articulan los
ámbitos económicos, social, cultural y ambiental para satisfacer las necesidades de las actuales
generaciones, sin poner en riesgo la satisfacción de necesidades de las generaciones futuras. La


exportador o generador, sobre quien además recaerá el costo de la gestión ambiental
correspondiente, sin perjuicio de las acciones civiles o penales a las que haya lugar.
TITULO VI
PRODUCCION Y CONSUMO SUSTENTABLE
CODIGO ORGANICO DEL AMBIENTE - Página 64
LEXIS FINDER - www.lexis.com.ec


El ejercicio del derecho humano al agua será sustentable, de manera que pueda ser ejercido por las
futuras generaciones. La Autoridad Unica del Agua definirá reservas de agua de calidad para el
consumo humano de las presentes y futuras generaciones y será responsa

## Preguntas y Respuestas con un modelo preparado para RAG

Ahora, utilizando el pipeline de RAG que hemos creado, y una LLM afinada especialmente para tareas de RAG, crearemos un Q&A.

Usaremos el modelo [llmware/dragon-deci-7b-v0](https://huggingface.co/llmware/dragon-deci-7b-v0). Un modelo de la serie DRAGON afinado especialmente para tareas relacionadas con extracción de información a partir de un contexto.

Los creadores del modelo han creado algunos conjuntos de datos para entrenar modelos en áreas específicas:
- **Dominios Enfocados:** Concentrándose en sectores como los servicios financieros, seguros, legal, cumplimiento y regulación.
- **Análisis de Contexto Cerrado:** Buscando respuestas derivadas de documentos de fuente específicos en lugar de conocimiento general.
- **Preguntas y Respuestas Basadas en Hechos:** Mejorando habilidades en la extracción de clave-valor, Q&A conciso, análisis básico, y resúmenes tanto de forma corta como larga.
- **Habilidades RAG Esenciales:** Construyendo conjuntos de entrenamiento dirigidos para Yes/No Booleano, reconocimiento de "no encontrado", matemáticas y lógica de sentido común, lectura de tablas y preguntas de elección múltiple.
- **Respuestas Claras y Concisas:** Enfocándose en respuestas breves para facilitar el manejo programático, correlación con fuentes de evidencia, riesgo reducido de alucinaciones, y procesamiento de inferencia más rápido.

Es un claro ejemplo de que el afinado de modelos pequeños en dominios y habilidades específicas, les permite rendir de formas similares a los modelos más generales y más grandes. Haciéndolos altamente efectivos y eficientes en costos en flujos de trabajo RAG y automatización relacionada en entornos de nube privada.


In [25]:
from langchain.prompts import ChatPromptTemplate
template = """<human>: Answer the question based only on the following context. Respond in Spanish. If you cannot answer the question with the context, please respond with 'I don't know':
### CONTEXT
{context}
### QUESTION
Question: {question}
\n
<bot>:
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from operator import itemgetter
import torch
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig, pipeline
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained("llmware/dragon-deci-7b-v0",
                                             trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained("llmware/dragon-deci-7b-v0",
                                          trust_remote_code=True)

generation_config = GenerationConfig(
    max_length=4096,
    temperature=1e-3,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)

pipeline = pipeline("text-generation",
                    model=model,
                    tokenizer=tokenizer,
                    max_length=4096,
                    temperature=1e-3,
                    do_sample=True,
                    eos_token_id=tokenizer.eos_token_id,
                    pad_token_id=tokenizer.eos_token_id
                    )

deci_dragon = HuggingFacePipeline(pipeline=pipeline)

Creamos el pipeline

In [19]:
retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | base_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | deci_dragon, "context": itemgetter("context")}
)

Testeamos el modelo

In [ ]:
###
question = "Describe evaluation criteria for retrieval augmented generation pipelines"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result['response'])

In [24]:
question = "¿Cuál es el procedimiento para expropiación de bienes?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result['response'])

/usr/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


Human: <human>: Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':
### CONTEXT
[Document(metadata={'author': '', 'creationdate': '2018-12-13T11:52:34-05:00', 'creator': '', 'keywords': '', 'moddate': '2019-01-24T19:26:48-06:00', 'page': 38, 'page_label': '39', 'producer': 'DynamicPDF for .NET v6.0.2.20 (Build 17136)', 'source': '/content/drive/MyDrive/Colab Notebooks/Proyecto Final/Pdfs/07_LY No. 0 CODIGO ORGANICO DEL AMBIENTE(983).pdf', 'subject': '', 'title': '', 'total_pages': 93}, page_content='reforma agraria, salvo cuando los fines sean exprópiatenos; y,\n2. De protección. Todo acto de ocupación ilegal por las vías de hecho o intento de despojo sin título\nalguno que habilite o demuestre la propiedad, es objeto de tutela administrativa, con efecto inmediato\nde desalojo de ocupantes y demolición, desmontaje o desmantelamiento de cualquier medio de\nafincamiento ilegal en progreso.\nArt. 

In [ ]:
question = "describes environmental rights"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result['response'])

## Creación de un dataset de evaluación

Podemos evaluar el modelo en formato `batch` o en `realtime`. Para hacerlo en formato batch necesitamos un dataset de preguntas y respuestas fiables para luego poder comparar el resultado de nuestro modelo con la respuesta esperada.

Este dataset lo podemos crear nosotros manualmente o... ¡usar un modelo más grande con altas capacidades para generarlo!

Usaremos GPT-3.5 para generar las preguntas y GPT-4 para contestarlas.

Finalmente, nuestro dataset deberá tener:

- **Preguntas:** Estos son los prompts que tu modelo RAG tratará. Asegúrate de que tu conjunto de datos incluya una amplia variedad de preguntas. Esta diversidad prueba la capacidad del modelo para manejar una amplia gama de temas y complejidades de preguntas.
- **Verdades Fundamentales:** Estas son las respuestas correctas a tus preguntas. Las utilizarás como referencia para medir con qué precisión responde tu modelo RAG.
- **Respuestas Predichas:** Estas son las respuestas que genera tu modelo RAG. Tu tarea clave es comparar estas respuestas con las verdades fundamentales para evaluar la precisión del modelo.
- **Contextos:** Estos proporcionan el antecedente o información suplementaria necesaria que tu modelo RAG utiliza para elaborar sus respuestas. Entender cómo tu modelo aprovecha este contexto es vital para evaluar su eficacia al incorporar información externa en sus respuestas.


In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

question_schema = ResponseSchema(
    name="question",
    description="a question about the context."
)

question_response_schemas = [
    question_schema,
]

In [ ]:
question_output_parser = StructuredOutputParser.from_response_schemas(question_response_schemas)

format_instructions = question_output_parser.get_format_instructions()

In [ ]:
question_generation_llm = ChatOpenAI(model="gpt-4o-mini")

bare_prompt_template = "{content}"

bare_template = ChatPromptTemplate.from_template(template=bare_prompt_template)

<ipython-input-28-ca61518e00c3>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  question_generation_llm = ChatOpenAI(model="gpt-4o-mini")


In [ ]:
from langchain.prompts import ChatPromptTemplate

qa_template = """\
  You are a University Professor creating a test for advanced students. For each context, create a question that is specific to the context. Avoid creating generic or general questions.
  question: a question about the context.
  Format the output as JSON with the following keys:
  question
  context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

messages = prompt_template.format_messages(
    context=docs[0],
    format_instructions=format_instructions
)

question_generation_chain = bare_template | question_generation_llm

response = question_generation_chain.invoke({"content" : messages})

output_dict = question_output_parser.parse(response.content)

In [ ]:
from langchain.prompts import ChatPromptTemplate

qa_template = """\
  You are an expert in Ecuadorian environmental legal regulations who creates frequently asked questions and answers for environmental professionals in Ecuador. For each context, create a question that is specific to the context. Avoid creating generic or general questions. Generate questions in Spanish
  question: a question about the context.
  Format the output as JSON with the following keys:
  question
  context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

messages = prompt_template.format_messages(
    context=docs[0],
    format_instructions=format_instructions
)

question_generation_chain = bare_template | question_generation_llm

response = question_generation_chain.invoke({"content" : messages})

output_dict = question_output_parser.parse(response.content)

In [ ]:
for k, v in output_dict.items():
  print(k)
  print(v)

question
¿Cuáles son las sanciones establecidas en el Código Orgánico del Ambiente para las actividades que no cumplan con las normativas ambientales vigentes?
context
CODIGO ORGANICO DEL AMBIENTE Ley 0 Registro Oficial Suplemento 983 de 12-abr.-2017


In [ ]:
from tqdm import tqdm
import random

random.seed(42)
qac_triples = []

loop = 5

for text in tqdm(random.sample(docs, loop)):

  messages = prompt_template.format_messages(
      context=text,
      format_instructions=format_instructions
  )

  response = question_generation_chain.invoke({"content" : messages})

  try:
    output_dict = question_output_parser.parse(response.content)
  except Exception as e:
    continue

  output_dict["context"] = text
  qac_triples.append(output_dict)

100%|██████████| 5/5 [00:17<00:00,  3.44s/it]


In [ ]:
for qac in qac_triples:
  print(qac)

{'question': '¿Cuáles son los criterios que la Autoridad Ambiental Nacional utilizará para determinar la inclusión de infraestructura en el plan de manejo del sistema de servicios ambientales?', 'context': Document(metadata={'producer': 'DynamicPDF for .NET v6.0.2.20 (Build 17136)', 'creator': '', 'creationdate': '2018-12-13T11:52:34-05:00', 'author': '', 'keywords': '', 'moddate': '2019-01-24T19:26:48-06:00', 'subject': '', 'title': '', 'source': '/content/drive/MyDrive/Colab Notebooks/Proyecto Final/Pdfs/07_LY No. 0 CODIGO ORGANICO DEL AMBIENTE(983).pdf', 'total_pages': 93, 'page': 23, 'page_label': '24'}, page_content='Territorial Nacional. El potencial de sus servicios ambientales será utilizado de manera sostenible\npara el desarrollo territorial y el bienestar de la población.\nLa Autoridad Ambiental Nacional analizará la inclusión dentro de suplan de manejo la construcción\nde infraestructura que sirva para la consecución de los fines del sistema.\nLas propiedades privadas cuya 

In [ ]:
answer_generation_llm = ChatOpenAI(model="gpt-4o", temperature=0)

answer_schema = ResponseSchema(
    name="answer",
    description="an answer to the question"
)

answer_response_schemas = [
    answer_schema,
]

answer_output_parser = StructuredOutputParser.from_response_schemas(answer_response_schemas)
format_instructions = answer_output_parser.get_format_instructions()

qa_template = """\
  You are an expert in Ecuadorian environmental legal regulations who creates frequently asked questions and answers for environmental professionals in Ecuador. For each question and context, create an answer. Generate questions and answers in Spanish.
  answer: a answer about the context.
  Format the output as JSON with the following keys:
  answer
  question: {question}
  context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)
messages = prompt_template.format_messages(
    context=qac_triples[0]["context"],
    question=qac_triples[0]["question"],
    format_instructions=format_instructions
)

answer_generation_chain = bare_template | answer_generation_llm
response = answer_generation_chain.invoke({"content" : messages})
output_dict = answer_output_parser.parse(response.content)

In [ ]:
for k, v in output_dict.items():
  print(k)
  print(v)

question
¿Cuáles son los criterios que la Autoridad Ambiental Nacional utilizará para determinar la inclusión de infraestructura en el plan de manejo del sistema de servicios ambientales?
answer
La Autoridad Ambiental Nacional evaluará la inclusión de infraestructura en el plan de manejo del sistema de servicios ambientales considerando su contribución a los objetivos del sistema. Esto implica que la infraestructura debe servir para el desarrollo territorial sostenible y el bienestar de la población, asegurando que el uso de los servicios ambientales sea sostenible. Además, se analizará si la infraestructura propuesta es necesaria para alcanzar los fines del sistema, lo que podría incluir aspectos como la mejora de la gestión de recursos naturales, la protección de la biodiversidad, o el apoyo a comunidades locales.
context
Territorial Nacional. El potencial de sus servicios ambientales será utilizado de manera sostenible para el desarrollo territorial y el bienestar de la población. L

In [ ]:
for triple in tqdm(qac_triples):

  messages = prompt_template.format_messages(
      context=triple["context"],
      question=triple["question"],
      format_instructions=format_instructions
  )

  response = answer_generation_chain.invoke({"content" : messages})

  try:
    output_dict = answer_output_parser.parse(response.content)
  except Exception as e:
    continue

  triple["answer"] = output_dict["answer"]

100%|██████████| 5/5 [00:33<00:00,  6.73s/it]


In [ ]:
import pandas as pd
from datasets import Dataset

ground_truth_qac_set = pd.DataFrame(qac_triples)
ground_truth_qac_set["context"] = ground_truth_qac_set["context"].map(lambda x: str(x.page_content))
ground_truth_qac_set = ground_truth_qac_set.rename(columns={"answer" : "ground_truth"})

eval_dataset = Dataset.from_pandas(ground_truth_qac_set)

In [ ]:
eval_dataset

Dataset({
    features: ['question', 'context', 'ground_truth'],
    num_rows: 5
})

In [ ]:
eval_dataset[0]

{'question': '¿Cuáles son los criterios que la Autoridad Ambiental Nacional utilizará para determinar la inclusión de infraestructura en el plan de manejo del sistema de servicios ambientales?',
 'context': 'Territorial Nacional. El potencial de sus servicios ambientales será utilizado de manera sostenible\npara el desarrollo territorial y el bienestar de la población.\nLa Autoridad Ambiental Nacional analizará la inclusión dentro de suplan de manejo la construcción\nde infraestructura que sirva para la consecución de los fines del sistema.\nLas propiedades privadas cuya titularidad del dominio sea anterior a la declaratoria del área',
 'ground_truth': 'La Autoridad Ambiental Nacional evaluará la inclusión de infraestructura en el plan de manejo del sistema de servicios ambientales considerando su contribución a los objetivos del sistema. Esto implica que la infraestructura debe servir para el desarrollo territorial sostenible y el bienestar de la población. Además, se analizará si la 

In [ ]:
from huggingface_hub import login
login()

In [ ]:
eval_dataset.push_to_hub("taniagdn/ragas-eval-dataset-amb")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/343 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/taniagdn/ragas-eval-dataset-amb/commit/2f12e090eb758cb6ba331b7ad1499d719bcac3a1', commit_message='Upload dataset', commit_description='', oid='2f12e090eb758cb6ba331b7ad1499d719bcac3a1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/taniagdn/ragas-eval-dataset-amb', endpoint='https://huggingface.co', repo_type='dataset', repo_id='taniagdn/ragas-eval-dataset-amb'), pr_revision=None, pr_num=None)

## RAG Evaluation using RAGas

Recordemos las métricas que se evalúan:
- **Relevancia de la Respuesta:** La pertinencia de la respuesta del modelo RAG al prompt dado.
- **Fidelidad:** Si las respuestas son fieles a los hechos proporcionados en el contexto.
- **Precisión del Contexto:** La capacidad del modelo para clasificar la información relevante al principio.
- **Corrección de la Respuesta:** La exactitud de la respuesta en comparación con la verdad objetiva.


In [ ]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness#,
    #context_recall,
    #context_precision,
    #answer_correctness,
    #answer_similarity
)
from ragas import evaluate

def create_ragas_dataset(rag_pipeline, eval_dataset):

  rag_dataset = []

  for row in tqdm(eval_dataset):
    answer = rag_pipeline.invoke({"question" : row["question"]})
    rag_dataset.append(
        {"question" : row["question"],
         "answer" : answer["response"],
         "contexts" : [context.page_content for context in answer["context"]],
         "ground_truths" : [row["ground_truth"]]
         }
    )

  rag_df = pd.DataFrame(rag_dataset)
  rag_eval_dataset = Dataset.from_pandas(rag_df)

  return rag_eval_dataset

def evaluate_ragas_dataset(ragas_dataset):

  result = evaluate(
    ragas_dataset,
    metrics=[
        #context_precision,
        faithfulness,
        answer_relevancy#,
        #context_recall,
        #answer_correctness,
        #answer_similarity
    ],
  )

  return result

Ahora ya podemos evaluar el dataset. Si quisiéramos evaluar solo un registro, lo haríamos con un solo registro ¿no?


In [ ]:
from tqdm import tqdm
import pandas as pd
basic_qa_ragas_dataset = create_ragas_dataset(retrieval_augmented_qa_chain, eval_dataset)

  0%|          | 0/5 [00:00<?, ?it/s]/usr/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
 20%|██        | 1/5 [00:06<00:27,  6.78s/it]/usr/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
 40%|████      | 2/5 [00:11<00:15,  5.30s/it]/usr/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
 60%|█

In [ ]:
basic_qa_result = evaluate_ragas_dataset(basic_qa_ragas_dataset)

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

## Resultados gráficos


In [ ]:
import matplotlib.pyplot as plt
def plot_metrics_with_values(metrics_dict, title='RAG Metrics'):
    """
    Plots a bar chart for metrics contained in a dictionary and annotates the values on the bars.
    Args:
    metrics_dict (dict): A dictionary with metric names as keys and values as metric scores.
    title (str): The title of the plot.
    """
    names = list(metrics_dict.keys())
    values = list(metrics_dict.values())
    plt.figure(figsize=(10, 6))
    bars = plt.barh(names, values, color='skyblue')
    # Adding the values on top of the bars
    for bar in bars:
        width = bar.get_width()
        plt.text(width + 0.01,  # x-position
                 bar.get_y() + bar.get_height() / 2,  # y-position
                 f'{width:.4f}',  # value
                 va='center')
    plt.xlabel('Score')
    plt.title(title)
    plt.xlim(0, 1)  # Setting the x-axis limit to be from 0 to 1
    plt.show()

In [ ]:
basic_qa_result

{'faithfulness': 0.7238, 'answer_relevancy': 0.1884}

In [ ]:
plot_metrics_with_values(basic_qa_result, "Base Retriever ragas Metrics")


AttributeError: 'EvaluationResult' object has no attribute 'keys'

- **Context Precision:** Esta métrica evalúa qué tan bien el sistema puede seleccionar información relevante del contexto proporcionado. Un valor alto indica que el sistema es capaz de distinguir y priorizar la información más relevante para la consulta.

- **Faithfulness:** Mide la fidelidad de las respuestas generadas respecto al contexto original. Una puntuación alta significa que la mayoría de la información presente en las respuestas puede ser rastreada de manera fiable al contexto, garantizando que las respuestas son factualmente consistentes.

- **Answer Relevancy:** Esta métrica determina qué tan relevantes son las respuestas a las preguntas formuladas. Valores altos indican que el sistema entiende bien la consulta y proporciona respuestas que se ajustan estrechamente a la necesidad de información del usuario.

- **Context Recall:** Evalúa la capacidad del sistema para recuperar toda la información relevante disponible en el contexto o base de datos para una consulta específica. Una puntuación alta aquí sugeriría que el sistema es muy eficiente en encontrar y utilizar toda la información pertinente.

- **Context Relevancy:** Esta métrica examina si el contexto recuperado y utilizado por el sistema para responder a una consulta es realmente pertinente para la pregunta hecha. Un valor bajo podría indicar que el sistema está recuperando mucha información que, aunque es relevante para el contexto en general, no es útil para la consulta específica.

- **Answer Correctness:** Mide la precisión o corrección de las respuestas dadas. Un valor moderadamente alto indica que una buena parte de las respuestas son correctas, pero también hay espacio para la mejora en la precisión de las respuestas.

- **Answer Similarity:** Esta métrica compara las respuestas generadas con las respuestas esperadas o ideales para ver qué tan cercanas son en términos de contenido y contexto. Un valor alto indica que las respuestas generadas por el sistema se asemejan mucho a las que se desearían o esperarían, mostrando una buena comprensión del problema.
